# Week 9 Exercise
## Victoria Hall
## DSC550
## 2.11.2022

### Step 1 - Importing Dataset

In [1]:
#loading packages and dataset
import pandas as pd

loan_df = pd.read_csv("Loan_Train.csv")

In [2]:
#Checking dataframe
loan_df.head(10)

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y
5,LP001011,Male,Yes,2,Graduate,Yes,5417,4196.0,267.0,360.0,1.0,Urban,Y
6,LP001013,Male,Yes,0,Not Graduate,No,2333,1516.0,95.0,360.0,1.0,Urban,Y
7,LP001014,Male,Yes,3+,Graduate,No,3036,2504.0,158.0,360.0,0.0,Semiurban,N
8,LP001018,Male,Yes,2,Graduate,No,4006,1526.0,168.0,360.0,1.0,Urban,Y
9,LP001020,Male,Yes,1,Graduate,No,12841,10968.0,349.0,360.0,1.0,Semiurban,N


### Step 2 - Preparing Data

In [3]:
#Dropping load column
loan_df = loan_df.drop(["Loan_ID"],axis=1)

In [4]:
#Checking for missing data
print("There are currently {} rows in the dataframe".format(loan_df.shape[0]))
print("Missing Values by Column")
print(loan_df.isnull().sum())


There are currently 614 rows in the dataframe
Missing Values by Column
Gender               13
Married               3
Dependents           15
Education             0
Self_Employed        32
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           22
Loan_Amount_Term     14
Credit_History       50
Property_Area         0
Loan_Status           0
dtype: int64


In [5]:
#Dropping cases with any missing data
loan_df.dropna(axis=0, inplace=True)

print("There are now {} cases in the dataframe".format(loan_df.shape[0]))

There are now 480 cases in the dataframe


In [6]:
#Looking at dtypes
print(loan_df.dtypes)

#Checking current number of columns
print("There are {} columns before the creation of dummy variables".format(loan_df.shape[1]))



Gender                object
Married               object
Dependents            object
Education             object
Self_Employed         object
ApplicantIncome        int64
CoapplicantIncome    float64
LoanAmount           float64
Loan_Amount_Term     float64
Credit_History       float64
Property_Area         object
Loan_Status           object
dtype: object
There are 12 columns before the creation of dummy variables


In [7]:
#Creating dummy variables for categorical objects in frame
#I don't want our target to have dummy features so I am going to manually chose the columns
loan_df = pd.get_dummies(loan_df, columns=["Gender","Married","Dependents","Education","Self_Employed","Property_Area"])

In [8]:
#Converting loan status to binary response
loan_df["Loan_Status"]=loan_df.Loan_Status.eq('Y').mul(1)

In [9]:
print("There are {} columns after the creation of dummy variables".format(loan_df.shape[1]))
loan_df.head(5)

There are 21 columns after the creation of dummy variables


,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Loan_Status,Gender_Female,Gender_Male,Married_No,Married_Yes,...,Dependents_1,Dependents_2,Dependents_3+,Education_Graduate,Education_Not Graduate,Self_Employed_No,Self_Employed_Yes,Property_Area_Rural,Property_Area_Semiurban,Property_Area_Urban
1,4583,1508.0,128.0,360.0,1.0,0,0,1,0,1,...,1,0,0,1,0,1,0,1,0,0
2,3000,0.0,66.0,360.0,1.0,1,0,1,0,1,...,0,0,0,1,0,0,1,0,0,1
3,2583,2358.0,120.0,360.0,1.0,1,0,1,0,1,...,0,0,0,0,1,1,0,0,0,1
4,6000,0.0,141.0,360.0,1.0,1,0,1,1,0,...,0,0,0,1,0,1,0,0,0,1
5,5417,4196.0,267.0,360.0,1.0,1,0,1,0,1,...,0,1,0,1,0,0,1,0,0,1


### Step 3 - Splitting into Train and Test

In [10]:
#importing package 
from sklearn.model_selection import train_test_split

#Creating features set
x = loan_df.drop('Loan_Status',axis=1)

#Creating target object
y = loan_df['Loan_Status']

#Creating training and test sets. 80% to train, 20% to test
x_train, x_test,y_train, y_test = train_test_split(x,y, test_size=.2)

### Step 4 - Creating a pipeline

In [11]:
#importing packages
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import GridSearchCV
import numpy as np
from sklearn import metrics
from sklearn.metrics import accuracy_score

In [12]:
#Creating the standardizer
min_max = MinMaxScaler()

#Creating KNN classifier. Setting n neighbors to 5 for initial model
knn = KNeighborsClassifier(n_neighbors=5, n_jobs= -1)

#Creating the pipeline
pipe = Pipeline([("min_max", min_max),("knn", knn)])

### Step 5 - Fit default classifer

In [13]:
#Fitting the base model to the training data
pipe.fit(x_train,y_train)

y_pred = pipe.predict(x_test)

#RMSE
print('RMSE of Test Data is {:.2}'.format(np.sqrt(metrics.mean_squared_error(y_test,y_pred))))

#R2
print('R2 of Test Data is {:.2}'.format(metrics.r2_score(y_test,y_pred)))

#Accuracy
print("The Accuracy score of the base model is {:.2%}:".format(accuracy_score(y_test,y_pred)))

RMSE of Test Data is 0.56
R2 of Test Data is -0.39
The Accuracy score of the base model is 68.75%:


### Steps 6 and 7 - Create a Search Space

In [14]:
#Creating search space with candidate values for n_neighbors of 1-10
search_space = [{"knn__n_neighbors": [1,2,3,4,5,6,7,8,9,10]}]


#Creating a grid search
classifier = GridSearchCV(pipe,search_space,cv=5, verbose=0).fit(x_train,y_train)

#Best number of neighborhood size
neighbors=classifier.best_estimator_.get_params()["knn__n_neighbors"]
print("{} is the best number of neighbors".format(neighbors))

3 is the best number of neighbors


### Step 8 - Find accuracy of grid search reccommended model

In [15]:
#Creating KNN classifier. Setting n neighbors to 3.
knn1 = KNeighborsClassifier(n_neighbors=3, n_jobs= -1)

#Creating the pipeline
pipe1 = Pipeline([("min_max", min_max),("knn", knn1)])



In [16]:
#Fitting the new model to the training data
pipe1.fit(x_train,y_train)

y_pred1 = pipe1.predict(x_test)

#RMSE
print('RMSE of Test Data is {:.2}'.format(np.sqrt(metrics.mean_squared_error(y_test,y_pred1))))

#R2
print('R2 of Test Data is {:.2}'.format(metrics.r2_score(y_test,y_pred1)))

#Accuracy
print("The Accuracy score of the updated model is {:.2%}:".format(accuracy_score(y_test,y_pred1)))

RMSE of Test Data is 0.53
R2 of Test Data is -0.25
The Accuracy score of the updated model is 71.88%:


The accuracy of our model increased slightly  when we used the recommended number of neighbors but our  R2 value was worsened showing we are accounting for less of the variance in the data with this model

### Step 9 - 11 - Checking other  models

In [17]:
#importing additional packages
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier


In [18]:


#starting with new pipeline
pipeline = Pipeline([('minmax', min_max), ('classifier',RandomForestClassifier())])

#Creating search space with additional learning algorithms
search_space2 = [{"classifier": [KNeighborsClassifier()], "classifier__n_neighbors" : [3]},
                {"classifier": [LogisticRegression()],"classifier__penalty": ['l1','l2'],"classifier__C": np.logspace(0,4,10)},
                {"classifier": [RandomForestClassifier()],"classifier__n_estimators": [10,100,1000],"classifier__max_features":[1,2,3]}]


#Creating grid search
gridsearch2 = GridSearchCV(pipeline, search_space2, cv=5,verbose=0)

#Fitting grid search
best_model = gridsearch2.fit(x_train,y_train)

#Reviewing best model
best_model.best_estimator_.get_params()["classifier"]

C:\Users\hallt\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:610: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\hallt\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\hallt\anaconda3\lib\site-packages\sklearn\pipeline.py", line 346, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "C:\Users\hallt\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 1306, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "C:\Users\hallt\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 443, in _check_solver
    raise ValueError("Solver %s supports only 'l2' or 'none' penalties, "
ValueError: Solver lbfgs supports only 'l2' or 'none' penalties, got 

C:\Users\hallt\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:610: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\hallt\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\hallt\anaconda3\lib\site-packages\sklearn\pipeline.py", line 346, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "C:\Users\hallt\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 1306, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "C:\Users\hallt\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 443, in _check_solver
    raise ValueError("Solver %s supports only 'l2' or 'none' penalties, "
ValueError: Solver lbfgs supports only 'l2' or 'none' penalties, got 

C:\Users\hallt\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:610: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\hallt\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\hallt\anaconda3\lib\site-packages\sklearn\pipeline.py", line 346, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "C:\Users\hallt\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 1306, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "C:\Users\hallt\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 443, in _check_solver
    raise ValueError("Solver %s supports only 'l2' or 'none' penalties, "
ValueError: Solver lbfgs supports only 'l2' or 'none' penalties, got 

C:\Users\hallt\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:610: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\hallt\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\hallt\anaconda3\lib\site-packages\sklearn\pipeline.py", line 346, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "C:\Users\hallt\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 1306, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "C:\Users\hallt\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 443, in _check_solver
    raise ValueError("Solver %s supports only 'l2' or 'none' penalties, "
ValueError: Solver lbfgs supports only 'l2' or 'none' penalties, got 

C:\Users\hallt\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:610: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\hallt\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\hallt\anaconda3\lib\site-packages\sklearn\pipeline.py", line 346, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "C:\Users\hallt\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 1306, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "C:\Users\hallt\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 443, in _check_solver
    raise ValueError("Solver %s supports only 'l2' or 'none' penalties, "
ValueError: Solver lbfgs supports only 'l2' or 'none' penalties, got 

C:\Users\hallt\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:918: UserWarning: One or more of the test scores are non-finite: [0.71353383        nan 0.7917635         nan 0.79172932        nan
 0.79172932        nan 0.79172932        nan 0.79172932        nan
 0.79172932        nan 0.79172932        nan 0.79172932        nan
 0.79172932        nan 0.79172932 0.68749146 0.74224197 0.75003418
 0.76045796 0.78909774 0.76558442 0.74743677 0.79166097 0.79688995]
  warnings.warn(


RandomForestClassifier(max_features=3, n_estimators=1000)

In [19]:
#Fitting the recommended Random Forest Classifier

#creating random forest classifer based on recommendations
rfc=RandomForestClassifier(max_features=3, n_estimators=1000)

#Creating new pipeline with random forest object
pipe3 = Pipeline([('minmax', min_max), ('classifier',rfc)])

pipe3.fit(x_train,y_train)

y_pred3 = pipe3.predict(x_test)

#RMSE
print('RMSE of Test Data is {:.2}'.format(np.sqrt(metrics.mean_squared_error(y_test,y_pred3))))

#R2
print('R2 of Test Data is {:.2}'.format(metrics.r2_score(y_test,y_pred3)))

#Accuracy
print("The Accuracy score of the best model is {:.2%}:".format(accuracy_score(y_test,y_pred3)))


RMSE of Test Data is 0.42
R2 of Test Data is 0.22
The Accuracy score of the best model is 82.29%:


The random forest classifier was determined to be the best model. It was the most accurate of the models we tested. However we did see a slight decrease of the R2 value, suggesting we are account for less of the variance in the model. 